### Version 1 - High Level Overview

In [12]:
from image_prep import get_img_df, train_val_test_split
from image_gen import get_gen_from_df

In [ ]:
fc_path = "data/Food_Classification/"

# get df
df_fc = get_img_df(fc_path)

num_classes = len(df_fc['label'].unique())
num_classes

In [14]:
# split dfs
train, validate, test = train_val_test_split(df_fc, test_size=0.2, val_size=0.2)

In [ ]:
train_gen = get_gen_from_df(train)
validate_gen = get_gen_from_df(validate, train=False)

The following model is probably not the final model we want to use but just needed something to test and end to end flow.  

This particular architecture was borrowed from the Jupyter notebook found: https://www.kaggle.com/code/varsha300/transferlearning


In [16]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Constants
IMG_SIZE = (224, 224)  # VGG16 default image size

# model

base_model = VGG16(weights='imagenet', include_top=False, input_tensor=Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3)))
# Freeze base model layers and unfreeze the last 4 layers
for layer in base_model.layers[:-4]:
    layer.trainable = False

# Model architecture
x = base_model.output
x = Flatten()(x)
x = Dense(4096, activation='relu')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
x = Dense(4096, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Compile model
model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Train model
history = model.fit(train_gen, validation_data=validate_gen, epochs=1, steps_per_epoch=train_gen.num_batches)


In [ ]:
# Evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(validate_gen)
print(f'Validation accuracy: {val_accuracy * 100:.2f}%')

### Version 2 - Find Best Model

In [1]:
import cv_models 

%load_ext autoreload
%autoreload 2

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 4060 Laptop GPU, compute capability 8.9


In [ ]:
"""
from image_prep import get_split_data
df_train, df_validate, df_test = get_split_data()
train_gen, validate_gen = cv_models.get_training_data(df_train, df_validate, batch_size=32)

type(train_gen)
"""


In [2]:
"""
Best Accuracy: 0.13067729771137238
Best validation set: Model_Xception__Dropout_0.3__HiddenSize_200

Validation accuracy: 13.78%
Best Accuracy: 0.1378486007452011
Best validation set: Model_Xception__Dropout_0.2__HiddenSize_500
"""

# store results in each run in case of exception
for best_acc, best_model_name, best_dropout, best_hidden, all_results in cv_models.find_best_model(5, batch_size=16):
    continue



**********
Training: Model_Xception__Dropout_0.2__HiddenSize_500**********

Found 3760 validated image filenames belonging to 20 classes.
Found 1255 validated image filenames belonging to 20 classes.


c:\Users\Jon\miniconda3\envs\Tasty_AI\lib\site-packages\keras\preprocessing\image.py:1139: UserWarning: Found 2 invalid image filename(s) in x_col="path". These filename(s) will be ignored.
  warnings.warn(


Epoch 1/5
117/117 [==============================] - 117s 974ms/step - loss: 7.9971 - accuracy: 0.0695 - val_loss: 8.8635 - val_accuracy: 0.0913
Epoch 2/5
117/117 [==============================] - 119s 1s/step - loss: 8.0787 - accuracy: 0.1062 - val_loss: 7.9392 - val_accuracy: 0.1178
Epoch 3/5
117/117 [==============================] - 116s 992ms/step - loss: 7.8204 - accuracy: 0.0974 - val_loss: 6.6486 - val_accuracy: 0.0745
Epoch 4/5
117/117 [==============================] - 113s 966ms/step - loss: 7.7928 - accuracy: 0.0885 - val_loss: 7.5460 - val_accuracy: 0.0553
Epoch 5/5
40/40 [==============================] - 26s 639ms/step - loss: 7.7490 - accuracy: 0.0741
Validation accuracy: 7.41%
Best Accuracy: 0.07410358637571335
Best validation set: Model_Xception__Dropout_0.2__HiddenSize_500
**********
Training: Model_Xception__Dropout_0.2__HiddenSize_600**********

Found 3760 validated image filenames belonging to 20 classes.
Found 1255 validated image filenames belonging to 20 class

c:\Users\Jon\miniconda3\envs\Tasty_AI\lib\site-packages\keras\preprocessing\image.py:1139: UserWarning: Found 2 invalid image filename(s) in x_col="path". These filename(s) will be ignored.
  warnings.warn(


Epoch 1/5
117/117 [==============================] - 116s 976ms/step - loss: 7.9379 - accuracy: 0.0840 - val_loss: 8.3504 - val_accuracy: 0.0697
Epoch 2/5
117/117 [==============================] - 115s 984ms/step - loss: 8.0164 - accuracy: 0.0998 - val_loss: 7.9142 - val_accuracy: 0.0946
Epoch 3/5
117/117 [==============================] - 114s 974ms/step - loss: 7.9094 - accuracy: 0.0805 - val_loss: 8.2673 - val_accuracy: 0.0938
Epoch 4/5
117/117 [==============================] - 115s 987ms/step - loss: 8.0824 - accuracy: 0.0891 - val_loss: 8.3780 - val_accuracy: 0.1066
Epoch 5/5
40/40 [==============================] - 26s 618ms/step - loss: 7.5472 - accuracy: 0.1124
Validation accuracy: 11.24%
Best Accuracy: 0.11235059797763824
Best validation set: Model_Xception__Dropout_0.2__HiddenSize_600
**********
Training: Model_Xception__Dropout_0.2__HiddenSize_800**********

Found 3760 validated image filenames belonging to 20 classes.
Found 1255 validated image filenames belonging to 20 c

c:\Users\Jon\miniconda3\envs\Tasty_AI\lib\site-packages\keras\preprocessing\image.py:1139: UserWarning: Found 2 invalid image filename(s) in x_col="path". These filename(s) will be ignored.
  warnings.warn(


Epoch 1/5
117/117 [==============================] - 118s 993ms/step - loss: 7.6744 - accuracy: 0.1025 - val_loss: 7.6173 - val_accuracy: 0.1731
Epoch 2/5
117/117 [==============================] - 115s 986ms/step - loss: 7.9561 - accuracy: 0.1062 - val_loss: 8.0239 - val_accuracy: 0.0921
Epoch 3/5
117/117 [==============================] - 115s 983ms/step - loss: 7.7978 - accuracy: 0.0955 - val_loss: 8.1924 - val_accuracy: 0.1258
Epoch 4/5
117/117 [==============================] - 116s 993ms/step - loss: 7.8217 - accuracy: 0.0810 - val_loss: 7.4915 - val_accuracy: 0.0585
Epoch 5/5
40/40 [==============================] - 26s 653ms/step - loss: 8.7476 - accuracy: 0.0805
Validation accuracy: 8.05%
**********
Training: Model_Xception__Dropout_0.2__HiddenSize_1000**********

Found 3760 validated image filenames belonging to 20 classes.
Found 1255 validated image filenames belonging to 20 classes.


c:\Users\Jon\miniconda3\envs\Tasty_AI\lib\site-packages\keras\preprocessing\image.py:1139: UserWarning: Found 2 invalid image filename(s) in x_col="path". These filename(s) will be ignored.
  warnings.warn(


Epoch 1/5


ResourceExhaustedError: Graph execution error:

Detected at node 'tasty_model_3/xception/block1_conv2/Conv2D' defined at (most recent call last):
    File "c:\Users\Jon\miniconda3\envs\Tasty_AI\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\Jon\miniconda3\envs\Tasty_AI\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\Jon\miniconda3\envs\Tasty_AI\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "c:\Users\Jon\miniconda3\envs\Tasty_AI\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "c:\Users\Jon\miniconda3\envs\Tasty_AI\lib\site-packages\ipykernel\kernelapp.py", line 739, in start
      self.io_loop.start()
    File "c:\Users\Jon\miniconda3\envs\Tasty_AI\lib\site-packages\tornado\platform\asyncio.py", line 205, in start
      self.asyncio_loop.run_forever()
    File "c:\

In [ ]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("TensorFlow is using GPU")
    for gpu in gpus:
        print("GPU:", gpu.name)
else:
    print("TensorFlow is not using GPU")